In [2]:
! pip install -git 'https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -g


In [ ]:
import JUSTFAIR_Toolbox as jt